### Concept of the app
1. Get Articles (once, weekly, daily, ...)

    a. scraping & parsing
    
    b. API access

    c. save to DB!!!

2. Recognise keywords and main topic
    
    a. TF-IDF
    
    b. ???

3. Group with other of the same topic
    
    a. Main topic matching
    
    b. Hierarchy (decision tree?)

4. Find divergences

    a. diff on keyword set

    b. difference of opinion

    c. difference of data sources

    d. difference in 

5. Represent articles and divergences

In [ ]:
import os
import sys
import time
import lxml
import json
import shutil
import codecs
import requests
import psycopg2
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

sys.path.append("../")

import utils.NYTParser
from utils.sql_utils import *
from utils.scrape_utils import *

In [ ]:
pipelineNYT(name='nyt_20200723_1600', loadDisk=True)
pipelineNYT(name='nyt_20200722_1000', loadDisk=True)
pipelineNYT(name='nyt_20200727_1200', fetchSource=True)

In [ ]:
pipelineAPINews(name='apinews_202027_1200', fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
all_sources = pd.read_sql("select * from newsdb.sources", getEngine())

In [ ]:
art = pd.DataFrame(articles)

In [ ]:
all_articles = pd.read_sql_table("articles", getEngine())
all_articles.shape